# Design

## Overall System Design

The SLAP (Self Logging Auto Pilot) system is designed to provide automated steering control for small sailing vessels while maintaining detailed logs of each journey. The system consists of several interconnected modules that work together to meet the system requirements and project objectives.

At its core, the system uses a PID (Proportional-Integral-Derivative) control algorithm that continuously monitors the boat's heading and makes adjustments to maintain the desired course. This is accomplished via a GPS sensor for postition and heading data. Additional sensors to detect sea and enviromental conditions. The sensor data and tiller adjustments are processed in real-time.

The user interface is implemented as a web application, allowing sailors to interact with the system through any mobile device with a web browser. This interface provides visualisation of key data like current heading whilst also allowing manual control inputs when needed. Communications between the web app and the smart phone is uses the RP5 WiFi HotSpot and a HTTP based communications.

Data logging is a key feature of the system, with detailed records of each journey stored in a database. The logs capture all sensor readings, including GPS position, motion sensor data, and heading. The logged data can be uploaded to a cloud based mapping application.

The system is designed to be reliable, it operates autonomously while still giving the sailor full control when needed.



## Features

Based on the *journey* and system objectives from the Analysis section, the necessary features to meet the overall requirements have been designed. The diagram below shows the system features arranged into groups.

![use cases](slapUseCases.drawio.png)


## UI Navigation and Design

Based on the user journey and features above, the following user interface structure was designed.

![userNavigationDesign](navigationDesign.drawio.png)


### UI Design

**Index:**  A central compass rose shows the angle of the target heading in degrees between 0 and 359. Central at the bottom is a start button which begins the Auto Pilot which will try to keep the boat on course. The stop button will stop the Auto Pilot control loop. Another button will start the logging. The simulate button will engage the boat simulator. Manual adjustments can be made with the buttons at the top, allowing for both fine and broad adjustments. The menu button in the top left will allows navigation between the pages.


**Sensors:** A listing page showing all the sensors including their names and their readings.

**Configs:** A listing page of all the system configurations with the facility to edit and delete each config.

**Config Form:** A form to enter config values, including a button to save or return.

**Trips:** A listiong page of recorded trips with a view button for each trip

**View Trip:** A page which presents details of a recorded trip.


## Modular Design

The system architecture follows a [modular design](https://www.youtube.com/watch?v=c_5pZYXCAuU), each module is designed to provide a clear set of functions. The webserver module, running MicroPython on an Rasberry Pi handles communication between the user interface and control systems. A dedicated PID module executes the main feedback control loop, receiving setpoint adjustments from the webserver when manual changes are made.

The design presented below was the result of a number of development iterations. The code for earlier iterations is included with the files submitted with this project

**Modular Design Diagram:**

![modularDesign](modularDesign.drawio.png)


## Module Descriptions and Interconnections

The SLAP system consists of modules that work together to provide automated steering control. Each module has specific responsibilities and functions. I have tried to make sure that related responsibilities have not been split between modules. The modules are described below. The code in the Technical Solution section is organised along the lines of these modules.

### Auto Pilot Module
- **PID Controller**: The central control module that implements the proportional-integral-derivative control algorithm
- **Input**: Current heading, desired heading, sensor data
- **Output**: Rudder position commands
- **Interfaces**: 
  - Recieves GPS data
  - Receives target heading updates from Web Interface
  - Sends control commands to Tiller Actuator

### Boat Dynamics Simulator Module
- **Components**: Virtual Boat Dynamics, Disturbances, Simulated geographic position calculations
- **Responsibilities**: For testing purposes by mimicking the behaviour of a boat
- **Interfaces**:
  - Recieves rudder angle
  - Calculates new heading and position
  - Provides simulated heading and position to GPS Module (in Sim Mode)

### Sensor Register Module
- **Components**: GPS, Motion Sensors, Eviroment Sensors
- **Responsibilities**: A register of all the hardware transducer modules and the sensors that they contain
- **Interfaces**:
  - Sends sensor data to Auto Pilot Controller
  - Keeps the GPS module updated
  - Provides data to Logging Module
  - Communicates with Web Interface for display

### GPS Module
- **Components**: Register of GPS values
- **Responsibilities**: To provide information about the boats position and heading
- **Interfaces**:
  - Collects data from either a GPS Hardware Module, the Magnetometer or the boat simulator
  - Supplies modules which require heading and positional data

### Tiller Actuator Module
- **Components**: Servo Motor Controller
- **Responsibilities**: Converts the tiller angle into signals for the servo motor
- **Interfaces**:
  - Receives tiller angle signal from PID Controller interface
  - Provides feedback to Web Interface

### Web Interface Module
- **Components**: Web Server, Javascripts, HTML Template for each page, style sheets
- **Responsibilities**: User interaction for system control and monitoring
- **Interfaces**:
  - Sends manual control inputs to Auto Pilot
  - Receives data from all sensors
  - Displays system status

### Logger Module
- **Components**: Data Logger
- **Responsibilities**: Collecting data and writing to database
- **Interfaces**:
  - Receives data from all sensors
  - Talks to SlapStore database service to store logs

### SlapStore Module
- **Components**: Database connection, Table Definitions, Storage Service Layer
- **Responsibilities**: Reading and writing to the database, keeping the database details hidden from the rest of the program
- **Interfaces**:
  - Connection to [SQLite database](https://www.tutorialspoint.com/sqlite/sqlite_python.htm)
  - Storage services for all parts of the system

### Mapping Module
- **Components**: Map Manager, Cloud interfacing
- **Responsibilities**: Read log data and write to the cloud mapping service
- **Interfaces**:
  - Receives data from SlapStore service layer
  - Talks to MapBox to upload trip logs

### Real-Time Behaviour
In early versions of SLAP, it proved too difficult to make the system function properly with a single program loop where each component is run in sequence. Therefore the system makes use of a number of execution threads. Early experimental code into threads can be found in the files submitted with the project. Each component which needs to execute in its own loop is given a dedicated thread. The choice of which modules required a thread was based on when a module needs to function in real-time.

The threaded modules are:
- Boat sim
- All sensors
- Logger
- Web server


### Communication Flow
1. The Sensor Modules continuously collects data
2. The Auto Pilot receives heading data from Sensors and calculates control values using the PID module
3. The Actuator Module recieves the control values from the Auto Pilot and controls the servo motor
4. The Logging Module records all sensor data at regular time interval
5. The Web Interface recieves user interactions, displays system status and sensor values
6. The Web Server, recieves interactions from the interface and sends them to the system modules
7. The Map Manager takes the Trip Log and uploadts it to the cloud mapping system


## Data and Code Design


### Data Dictionary

The following data items are used throughout the system:

| Data Item | Description | Format | Example |
| --- | --- | --- | --- |
| Heading | The current compass direction the boat is pointing | float (0-359 degrees) | 180 |
| Target Heading | The desired compass direction for the boat | float (0-359 degrees) | 175 |
| Heading Error | Difference between current and target heading | float (degrees) | -5 |
| Position | Current location of the boat | string (latitude, longitude) | (50.7192, -1.8808) |
| Rudder Position | Current rudder angle | float (-45 to +45 degrees) | -10 |
| Servo Output | Signal sent to rudder servo | float (-1.0 to 1.0) | 0.55 |
| Log Time | Timestamp for data logging | DateTime | 2024-01-20 14:30:00 |
| Trip ID | Unique identifier for each journey | Integer | 1234 |


### Primary Sensor Data

The system uses the following sensors:

 - BNO055 9-axis IMU sensor for:

   - Heading data (compass)

   - Roll and pitch angles

   - Acceleration

 - NEO-6M GPS module for:

   - Position (latitude/longitude)

   - Ground speed

   - Course over ground

 - DS18B20 temperature sensor for:

   - Water temperature monitoring

 - INA219 current/voltage sensor for:

   - Battery voltage monitoring

   - Current draw measurement


## Data Stucture Definitions


### Database Design

The diagram below represents the database entities in the system, their relationships and the multiplicities.

![erd](erd.drawio.png)

The database consists of four main tables:


Config Table:

- Contains config used to tune the control algorithm

- Each config has a name and three value

- config include PID control values

- Multiple config sets can be stored for different sea conditions


Sensors Table:

- Maintains a registry of all sensors connected to the system

- Each sensor has a unique Identifier, name and units

- Sensor types include GPS, compass, direction, heading, accelerometer etc.


Trips Table:

- Records details of each sailing trip

- Contains start/end times

- Links to config settings used for that trip


Readings Table:

- Stores all sensor readings during trips

- Each reading links to a specific trip and sensor

- Includes timestamp and raw sensor value

- One reading record per regular interval



### Data Validation

Data entered into input fields in the web based UIs, such as editing configs, must be validated to ensure they are of the of the correct format and type. Being an active with multiple hardware sensors, most of the other data in the system is recieved automatically and will always is valid. The limited user input data is validated in the user interface. The following user interfaces capture data and therefore be validated as indicated below:

| Form | Input Field | Data Type | Valid Range/Format | Description |
| ------------- | ------------- | ------------- | ------------- | ------------- |
| Index | Set Heading | Integer | 0 to 359 degrees | Desired boat heading |
| Edit Config | Config Name | Text | 15 characters max | Name of configuration |
| Edit Config | P Value | Float | Positive decimal | Proportional control value |
| Edit Config | I Value | Float | Positive decimal | Integral control value |
| Edit Config | D Value | Float | Positive decimal | Derivative control value |


Other user input data does require validation as it is contrained by the input method. For example + / - 1 or 10 degree buttons. These inputs do not allow invalid data input



### Object Oriented Programming

The design of the code for using hardware modules each with a number sensors is complex. The following design was arrived at after a number of iteration. It is explained here due to its complexity. To achieve a clean code solution a class inheritance pattern is utilised.

The diagram below shows the object oriented class structures used:

![sensorStructure](sensorClassStruct.drawio.png)

The structure uses two abstract base classes. This allows the system code to work with specific Tranducer Modules and their associated Sensors without needing to know their specific details.

The `Transducer` abstract class represents hardware devices. Note that each hardware device can contain a number of actual sensors. 

The Transducer class has:

- A list of sensors

- Thread management so that the readings from the hardware can be collected in real-time

The `Sensor` abstract class represents the sensors in each hardware (Transducer) module, including:

- basic sensor properties (name, units)

- and, getValue method to retrieve sensor reading


Specific sensor class which inherit from `Sensor` and represent real sensors:

- `heading`: The direction of travel in degrees

- `position`: The longitude and latitude

- `temperature`: The enviroment's temperature

- `pressure`: Air pressure 

This [object oriented class structure](https://www.datacamp.com/tutorial/python-oop-tutorial) allows the rest of the system to handle sensor information without having to handle their specific types. It allows a register of sensors to be created.

The implementations of the Transducer classes contain the code necessary to interface the hardware with the RPi5. The implementation of the Sensor classes defines the nature of all the sensors in the system.



## Algorithms

There are some key algorithms within the system which required some design. The control algorithm was first investigated as part of the Analysis section. These algorithms are explained in detail below.

### Boat Simulator

The boat simulator allows SLAP to run without the need of a real boat, this is necessary for test and development purposes, and understanding the boat dynamics and creating modelling code is a key part of the project. Without the simulator the objective of a prototype system ready for testing on a real boat cannot be met.

There are two element to the boat simulator.The first is the mathmatical model which represents the boats response to the tiller. This is characterised by calculating the new heading using the previous information about the boats heading. The dynamics of this turning behaviour are that the boat begins to turn quickly, as the boat turns its rate of turn slows until a steady rate is reached. This is a [first order differential response](https://www.youtube.com/watch?v=aLktwqbgUXE. i.e. the rate of change of turn is proportional to the difference (differential) between the current and eventual rate of turn. 

The second element of the boat modeling is the disturbances to the boats heading due to the enviroment. The boat simulator would not be realistic without this as the control algorithm would simply and quickly achieve the correct heading. The boat simulator model therefore randomly generates disturbances which affects the boats heading and forces the Auto Pilot control algorithm to react.

**Mathmatics**

In order to control the movement and direction of the boat we must first be able to model how the rudders angle affects the boats direction.

We understand that the rudders angle and boat's direction do not respect a linear relationship, if we represent the rudder's angle with $\theta$, and the turning radius of the boat as $R$ we can see that: $R = \frac{1}{K |\theta|}$ where $K$ is some constant .

This inversely proptional relationship is proven as we know as the rudders angle increases, the turning radius decreases. The rate of which the boats heading changes, also known as the angular velocity, $\omega$, can be represented as the differential of the angle and time: $\frac{dA}{dt}$ or as the velocity over the radius: $\frac{v}{R}$.

Therefore $\omega = \frac{v}{(K|\theta|)^{-1}}$, and so: $$\omega = vK|\theta|$$

We can simply intergrate this to find the change in angle over time to give : $$\Delta\omega = vK|\theta|t$$

However we understand that the boat does not turn to the target radius immediately, there is always some lag between turning the rudder and the target radius being reached.


The turn rate follows an exponential decay until it reaches the target, the standard decay equation is $N_{t+1} = N_{t} - e^{-\lambda t}$

Where lambda is our decay constant and $t$ is our discrete time value.


This can be implemented with our boat applicable values: $$\omega(t) = \omega_{\infty} \left( 1 - e^{-t/\tau} \right)$$

This algorithm is implemented in code as descibed in the Technical Solution section(boatSim.py)

**Disturbances**

The disturbance algorithm randomly picks a start time by using the current time and adding a random value. Its duration is determined by repeating this process but using the start time. When a disturbance is inflicted on the simulator its rate of turn is increased or decreased by a random magnitude. All random values are determined within a realistic range to mimic the affect of sea conditions on a boat.

The following code shows the disturbance algorithms:

In [1]:
TIMECONSTANT = 0.333333
MAX_DISTURBANCE_DURATION = 5000 # ms
MIN_DISTURBANCE_DURATION = 2000 # ms
MAX_DISTURBANCE_MAGNITUDE = 20 # degrees per secon


# Create a disturbance based on a random start time, duration and magnitude
def createDisturbance(self):
        currentTimeMilli = int(round(time.time() * 1000))
        
        disturbance =  random.randint(-MAX_DISTURBANCE_MAGNITUDE, MAX_DISTURBANCE_MAGNITUDE)
        
        startTime = currentTimeMilli + random.randint( MAX_DISTURBANCE_DURATION, 2 * MAX_DISTURBANCE_DURATION)
        
        endTime = startTime + random.randint(MIN_DISTURBANCE_DURATION,MAX_DISTURBANCE_DURATION)
        
        output = {'endTime': endTime,
                'disturbance': disturbance,
                'startTime': startTime}
        
        return output
    
# Return the disturbance if it is currently active, otherwise create a new one
def disturbance(self):
        currentTimeMilli = int(round(time.time() * 1000))
        if (currentTimeMilli < self.nextDisturbance['startTime']):
            return 0
        elif currentTimeMilli > self.nextDisturbance['startTime'] and currentTimeMilli < self.nextDisturbance['endTime']:
            return self.nextDisturbance['disturbance']
        else:
            self.nextDisturbance = self.createDisturbance()
            return 0



### PID Control Algorithm

The [PID control algorithm](https://medium.com/@aleksej.gudkov/python-pid-controller-example-a-complete-guide-5f35589eec86) allows SLAP to adjust the tiller angle to correct its course using the difference between its actual heading and the desired heading. During the Analysis it was discovered that a control algorithm known as PID was well suited to this project. The figure below shows the PID algorithm in diagramatic form:

![pidDiagram](pidDiagram1.drawio.png)

**Mathmatics**

The PID (Proportional, Integral, Derivative) control algorithm is a feedback control system that calculates an error value as the difference between a desired setpoint and a measured process variable. The algorithm applies three types of control:

1. Proportional ($P$): Responds proportionally to the current error
2. Integral ($I$): Accumulates past errors to eliminate steady-state error
3. Derivative ($D$): Predicts future error based on its rate of change

The output is calculated as:
$output = K_p e_t + K_i \int e_t dt + K_d \frac{de}{dt}$

Where:
- $K_p, K_i, K_d$ are tuning parameters
- $e_t$ is the error at time $t$
- The integral term sums past errors
- The derivative term predicts future errors

This algorithm is implemented in SLAP to maintain the boat's heading by continuously adjusting the rudder angle based on the difference between the desired and actual heading.


The PID method shown in the code below and was used in the Analysis section to investigate suitable control algorithms. The full PID algorithm used in the SLAP is detailed in the Technical Solution section.

In [2]:
# %load slap/src/pid/experimentForReport/pidModule.py

class PidController:

    def __init__(self,KP,KI,KD):
        self.kp = KP
        self.ki = KI
        self.kd = KD
        self.accumlatedError = 0
        self.lastPos = 0
        self.elapsed = 0

    def pid(self, pos, target, dt):

        # PROPORTIONAL-------
        error = target - pos
        proportional = error

        # Intergral----------
        intergal = ((error * dt) + self.accumlatedError)
        self.accumlatedError = intergal


        # Differential-------
        dpos = self.lastPos - pos
        differential = (dpos / dt)

        self.lastPos = pos

        return self.kp * proportional + self.ki * intergal + self.kd * differential

### Working with Compass Headings

There are complications in the arithmatic relating to compass headings. This is due to the fact a compass functions on a 360 degree basis. For example the difference between a heading of 5 degrees and 355 degrees should be -10 degrees and 355 degrees + 10 degrees must equal 5 degrees not 365 degrees.

When calculating which way to adjust the tiller as a part of the control algorithm, this arthmetic complication is important otherwise the boat will attempt to correct its course the wrong way around. The algorithms for these calculations are shown in the code snippets below.

It took some time to get the system to work due to these complications. Early versions of the code included development of these algorithms. These versions can be found in the file submitted for this project

The basic approach to dealing with compass headings is to use modulo 360 arithmetic. Secondly when calculating differences for control we must find the smallest change in heading necessary.

In [3]:

# Get the error between the target and heading in its shortest path
def getHeadingError(self, target, heading):
    if target - heading <= 180:
        error = target - heading
    else:
        error = (target - heading) - 360
    return error

# Convert a heading to a value between 0 and 359
def compassify(input):
    print("Compassing: ", input)
    
    return input% 360


## Security

In designing the security for this project, the first things to consider are the risks. The main risk identified is the boat steering incorrectly. The most likely reason for this is the system preforming incorrectly. To reduce this risk, the system must be tested rigorously.

The second threat identified is inappropiate access to the system. If the system were connected to the internet, during operation, the risk would be significant. However, on a boat with SLAP connected to a smart phone via a WiFi HotSpot the oppotunity for an unorthorised user to access the system is very small. This is because access is controlled by the WiFi security on the RPi5 HotSpot.


## Testing Strategy


The testing strategy for SLAP involves both manual and unit testing approaches. The testing strategy is designed to ensure the system includes all features and meets the specific project objectives.

### Manual Testing
Manual testing will be conducted to verify the physical operation of the system. This includes:
- Testing the tiller control mechanism
- Verifying compass readings
- Checking the user interface functionality

### Unit Testing
Unit tests will be developed for each major component of the system


### General Feature Testing
Each identified function from the Analysis section will be tested


### Project Objectives Testing
The testing strategy ensures all project objectives are met.


